# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = 'city_data_df.csv'
data = pd.read_csv(file)
data


,City Name,Lat,Lng,Max Temp,Humidity,Clouds,Wind,Country,Date
0,mnogovershinnyy,53.9353,139.9242,54.41,90,100,2.53,RU,1628054974
1,bambous virieux,-20.3428,57.7575,70.21,94,75,6.91,MU,1628054975
2,kieta,-6.2167,155.6333,82.72,79,32,6.31,PG,1628054975
3,narsaq,60.9167,-46.0500,44.82,98,100,3.36,GL,1628054976
4,hermanus,-34.4187,19.2345,51.91,87,60,5.17,ZA,1628054976
...,...,...,...,...,...,...,...,...,...
560,sampit,-2.5333,112.9500,91.90,46,95,3.29,ID,1628055246
561,homer,59.6425,-151.5483,62.49,75,1,6.91,US,1628055246
562,seddon,-37.8068,144.8916,60.08,76,90,7.00,AU,1628055246
563,belmonte,-15.8631,-38.8828,66.70,81,43,7.61,BR,1628055247


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
fig = gmaps.figure()
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
gmaps.configure(api_key=g_key)
locations = data[['Lat', 'Lng']]
humidity = data['Humidity']
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating=False, max_intensity=100,
                                point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
hotel_df = data.loc[(data['Max Temp'] < 80) & (data['Max Temp'] > 70) & (data['Wind'] < 10) & (data['Clouds'] == 0)].copy()

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:

hotel_df['Hotel Name'] = ''

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
        "radius": 5000,
        "type": "hotel",
        "key": g_key}

for index, row in hotel_df.iterrows():
    
    params['location'] = f"{row['Lat']}, {row['Lng']}"


    response = requests.get(base_url ,params=params)
    results = response.json()
    
    hotel_df.loc[index, 'Hotel Name']= results['results'][0]['name']
hotel_df.rename(columns={'City Name': 'City'}, inplace=True)

hotel_df

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind,Country,Date,Hotel Name
104,kabinda,-6.1333,24.4833,74.17,33,0,6.20,CD,1628055037,Kabinda
192,eskisehir,39.7767,30.5206,71.47,64,0,8.05,TR,1628055081,Eskişehir
345,moroni,-11.7022,43.2551,75.96,63,0,9.95,KM,1628055147,Moroni
418,marovoay,-16.1000,46.6333,73.54,45,0,9.10,MG,1628055180,Marovoay
518,orumiyeh,37.5527,45.0761,76.55,41,0,4.61,IR,1628055227,Urmia


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))